In [2]:
import pandas as pd

In [4]:
df=pd.read_csv('/content/cleaned_and_filtered.csv')

In [5]:
!pip install transformers torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [10]:
from transformers import pipeline
import pandas as pd

# Load your data
df = pd.read_csv('/content/cleaned_and_filtered.csv')

# Load the sentiment and emotion pipelines
sentiment_analysis = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment", device=0)  # device -1 for CPU
emotion_analysis = pipeline("text-classification", model="bhadresh-savani/distilbert-base-uncased-emotion", device=0)

# Function to apply the model in batches
def apply_model_in_batches(model, texts, batch_size=100):
    results = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        results.extend(model(batch))
    return results

# Apply sentiment analysis in batches
sentiment_results = apply_model_in_batches(sentiment_analysis, df['TITLE'].tolist())
df['sentiment'] = [result['label'] for result in sentiment_results]
df['sentiment_score'] = [result['score'] for result in sentiment_results]

# Apply emotion analysis in batches
emotion_results = apply_model_in_batches(emotion_analysis, df['TITLE'].tolist())
df['emotion'] = [result['label'] for result in emotion_results]
df['emotion_score'] = [result['score'] for result in emotion_results]

# Save the DataFrame


In [11]:
def label_to_sentiment(label):
    mapping = {
        'LABEL_0': 'negative',
        'LABEL_1': 'neutral',
        'LABEL_2': 'positive'
    }
    return mapping.get(label, 'unknown')  # Return 'unknown' if the label is not recognized

# Apply the function to the 'sentiment' column
df['sentiment'] = df['sentiment'].apply(label_to_sentiment)

# Optionally save the updated DataFrame back to a CSV
df.to_csv('main.csv', index=False)

# Print the head of the DataFrame to verify changes


In [12]:
df.head()

,Unnamed: 0,DATE,TITLE,AUTHOR,ARTICLE,content,sentiment,sentiment_score,emotion,emotion_score
0,657,2019-06-29,"Sanchez, Nationals shut down Tigers",Field Level Media,Anibal Sanchez pitched six strong innings Frid...,NaN,neutral,0.892794,anger,0.772276
1,1055,2019-06-11,Rays top A's as Morton's unbeaten streak hits 21,Field Level Media,"Aided by three home runs, right-hander Charlie...",NaN,neutral,0.626774,joy,0.990553
2,1235,2019-06-15,"Jimenez, White Sox crush Sabathia, Yankees",Field Level Media,EditorsNote: Changed stat ‘eight runs’ to ‘fiv...,NaN,neutral,0.899590,anger,0.657171
3,1374,2019-06-15,"Giants crack three homers, down Brewers",Field Level Media,Kevin Pillar had three hits and drove in the t...,NaN,neutral,0.829722,anger,0.467322
4,1408,2019-06-19,"A's crack six homers, obliterate Orioles",Field Level Media,Beau Taylor triggered a six-homer assault on B...,NaN,neutral,0.706669,anger,0.490198
